In [16]:
import tensorflow as tf
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageSequence
import cv2
%matplotlib inline

In [ ]:
class gifEmbedding():
    def __init__():
        self.inputs=  tf.placeholder(tf.float32,(None,63,256,256,1))
    def model():
        return tf.layers.conv3d(self.inputs,filters = 64 ,kernel_size=(3,3,3))
    def randomRun(frames):
        with tf.Session() as sess :
            sess.run(tf.global_variables_initializer())
            out =  sess.run(model, feed_dict={self.inputs:frames})
        return out

In [17]:
gif1 = Image.open('sample.gif')
frames = np.array([cv2.resize(np.array(frame.copy()),(256,256)) for frame in ImageSequence.Iterator(gif1)]).reshape(1,-1,256,256,1)
gif2 = Image.open('sample2.gif')
frames2 = np.array([cv2.resize(np.array(frame.copy()),(256,256)) for frame in ImageSequence.Iterator(gif2)]).reshape(1,-1,256,256,1)
gif3 = Image.open('sample3.gif')
frames3 = np.array([cv2.resize(np.array(frame.copy()),(256,256)) for frame in ImageSequence.Iterator(gif3)]).reshape(1,-1,256,256,1)
print frames.shape,frames2.shape,frames3.shape

(1, 63, 256, 256, 1) (1, 23, 256, 256, 1) (1, 27, 256, 256, 1)


In [22]:
sample = 


In [23]:
out.shape

(1, 61, 254, 254, 64)